importing dependencies

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score



data collection and preprocessing

In [3]:
# loading the data from csv file to a pandas dataframe
raw_mail_data=pd.read_csv('/content/mail_data.csv')

In [4]:
print (raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [9]:
raw_mail_data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [11]:
raw_mail_data.isnull().sum()

,0
Category,0
Message,0


In [13]:
print(raw_mail_data.isnull())

      Category  Message
0        False    False
1        False    False
2        False    False
3        False    False
4        False    False
...        ...      ...
5567     False    False
5568     False    False
5569     False    False
5570     False    False
5571     False    False

[5572 rows x 2 columns]


In [14]:
# replace null values with a null string
mail_data=raw_mail_data.where((pd.notnull(raw_mail_data)),'')

In [15]:
mail_data.shape

(5572, 2)

Label Encoding

In [17]:
# label spam mail as 0 and ham mail as 1

mail_data.loc[mail_data['Category']=='spam','Category',]=0
mail_data.loc[mail_data['Category']=='ham','Category',]=1

In [21]:
# seperating the data as text and label

X=mail_data['Message']
Y=mail_data['Category']

In [22]:
print(X)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object


In [23]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


In [27]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=3)

Feature Extraction

In [42]:
# transform the text data to feature vectors that can be used as input to the logistic regression

feature_extraction=TfidfVectorizer(min_df=1,stop_words='english',lowercase=True)

In [43]:
X_train_features=feature_extraction.fit_transform(X_train)
X_test_features=feature_extraction.transform(X_test)


# convert Y_train and Y_test values as integers

Y_train=Y_train.astype('int')
Y_test=Y_test.astype('int')


In [ ]:
print(X_train_features)

training model

using logistic regression model

In [45]:
model=LogisticRegression()

In [46]:
# training the logistic regression model with the training data

model.fit(X_train_features,Y_train)

LogisticRegression()

model evaluation

In [48]:
predict_on_training=model.predict(X_train_features)

In [49]:
# accuracy on training data

accuracy_score_on_training=accuracy_score(Y_train,predict_on_training)

In [50]:
print("Accuracy on training data: ",accuracy_score_on_training)

Accuracy on training data:  0.9676912721561588


prediction on test data

In [51]:
predict_on_test_data=model.predict(X_test_features)

accuracy_score_on_test=accuracy_score(Y_test,predict_on_test_data)

In [52]:
print("Accuracy on test data: ",accuracy_score_on_test)

Accuracy on test data:  0.9668161434977578


In [59]:
from sklearn.metrics import confusion_matrix, classification_report

# Predictions already stored in X_test_prediction
cm = confusion_matrix(Y_test, predict_on_test_data)
print("Confusion Matrix:\n", cm)

print("\nClassification Report:\n", classification_report(Y_test, predict_on_test_data))


Confusion Matrix:
 [[118  37]
 [  0 960]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.76      0.86       155
           1       0.96      1.00      0.98       960

    accuracy                           0.97      1115
   macro avg       0.98      0.88      0.92      1115
weighted avg       0.97      0.97      0.96      1115



Building a predictive

In [62]:
def predict_mail(mail):
    # Transform the email using the same TF-IDF vectorizer
    mail_features = feature_extraction.transform([mail])

    # Predict using trained model
    prediction = model.predict(mail_features)

    if prediction[0] == 0:
        return "Spam 🚫"
    else:
        return "Ham ✅"



In [63]:
# 🔍 Example Tests
print(predict_mail("You won $1000, claim now!"))
print(predict_mail("Let's meet at 5 pm today."))


Spam 🚫
Ham ✅


testing from the data set

In [64]:
print(predict_mail("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's"))

Spam 🚫


In [65]:
print(predict_mail("As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune"))

Ham ✅
